In [17]:
# ==========================================
# 1. Import required libraries
# ==========================================
import pandas as pd 
import os
from sqlalchemy import create_engine
import time
import logging

# ==========================================
# 2. Configure logging
# ==========================================
# Make sure "logs" folder exists
os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    filename="logs/ingestion_db.log",   # log file path
    level=logging.DEBUG,                # log level
    format="%(asctime)s - %(levelname)s - %(message)s", 
    filemode="a"                        # append mode
)

# ==========================================
# 3. Create SQLite Database Connection
# ==========================================
engine = create_engine('sqlite:///Ecommerce.db')

# ==========================================
# 4. Load CSV dataset into DataFrame
# ==========================================
df = pd.read_csv(r"C:\Users\Dell\Downloads\ecommerce_sales_dataset_large.csv")

# Preview dataset
print(df.head())

# Dataset info
print(df.info())
print(df.isnull().sum())
print("Duplicates:", df.duplicated().sum())

# Number of rows and columns
print("Shape of dataset (rows, columns):", df.shape)
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])

# ==========================================
# 5. Function to ingest DataFrame into SQLite DB
# ==========================================
def ingest_db(df, table_name, engine):
    """
    Function to store a DataFrame into a SQLite database table.
    
    Parameters:
    df (DataFrame): The pandas DataFrame to store.
    table_name (str): Name of the SQL table.
    engine (Engine): SQLAlchemy engine object.
    """
    try:
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"✅ Data successfully ingested into table '{table_name}'")
        logging.info(f"Data successfully ingested into table '{table_name}'")
    except Exception as e:
        print(f"❌ Error ingesting data into table '{table_name}': {e}")
        logging.error(f"Error ingesting data into table '{table_name}': {e}")

# ==========================================
# 6. Ingest loaded dataset into DB
# ==========================================
ingest_db(df, "EcommerceSales", engine)

# ==========================================
# 7. Function to ingest all CSV files in 'data/' folder
# ==========================================
def load_raw_data():
    """
    This function will load all CSV files in 'data' folder 
    and ingest them into SQLite database.
    """
    start = time.time()
    
    # Ensure data folder exists
    if not os.path.exists('data'):
        os.makedirs('data')
        logging.warning("'data' folder was missing. Created an empty one.")
    
    for file in os.listdir('data'):
        if file.endswith('.csv'):
            try:
                df = pd.read_csv(os.path.join('data', file))
                logging.info(f"Ingesting {file} into DB...")
                ingest_db(df, file[:-4], engine)  # table name without '.csv'
            except Exception as e:
                logging.error(f"Failed to ingest {file}: {e}")
    
    end = time.time()
    total_time = (end - start) / 60
    logging.info(".......Ingestion complete........")
    logging.info(f"Total time taken: {total_time:.2f} minutes")
    print("✅ All CSV files from 'data/' folder ingested successfully!")

# ==========================================
# 8. Run load_raw_data() if needed
# ==========================================
# load_raw_data()




   OrderID   Product        Category Region CustomerType       Month  \
0        1     Jeans        Clothing   West       Repeat  2023-06-30   
1        2     Mixer  Home & Kitchen   West       Repeat  2024-04-30   
2        3     Chair  Home & Kitchen  South          New  2024-10-31   
3        4  Textbook           Books  South          New  2024-01-31   
4        5  Lipstick          Beauty  North          New  2023-06-30   

   UnitPrice  Quantity  Revenue  
0        388         8     3104  
1        414         1      414  
2        194         7     1358  
3        181         4      724  
4        301         3      903  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   OrderID       100000 non-null  int64 
 1   Product       100000 non-null  object
 2   Category      100000 non-null  object
 3   Region        100000 non-null  obj

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Reconnect to DB
engine = create_engine('sqlite:///Ecommerce.db')

# Load all data from EcommerceSales table
df_all = pd.read_sql("SELECT * FROM EcommerceSales", con=engine)

# Show first 5 rows
print(df_all.head())

# Show total rows & columns
print("Shape of dataset:", df_all.shape)

# If you want to see entire dataset (⚠️ careful with very large data)
# print(df_all.to_string())   # This will print ALL rows (not recommended for 1L+ rows)

# Better option: display only first 100 rows
print(df_all.head(100))
